In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = False
discrete = False
simp = True # let's leave this true for now because of the way we calculate the piecewise discrete "derivative" below. Seems to result in vastly simpler equations.

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [4]:
def display_large(expr):
  latex_code = f"\\large {{ {latex(expr)} }}"
  display(Math(latex_code))

### TODO either show convexity for everything or verify optimality otherwise (i.e. by simply plugging into offchain-tests)

## Target function

we want to minimize the effective price, given a set of exponents for the buying
and selling asset each.

In [5]:
if delta_b_negative:
  eff = Delta_s / -Delta_b
else:
  eff = Delta_s / Delta_b
eff

Delta_s/Delta_b

## Equalities

In [6]:
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if delta_b_negative and asset == 'b':
    f = -f
  if discrete:
    f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

-b_b + s_b/w_b - v_b

In [7]:
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if discrete:
    f = floor(f)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

## Equality-constraints

In [8]:
def deltaSpotBound_(asset, s,v, b, w, j, e, Delta, a, min):
  eq = deltaBySpot[asset] - Delta # ==! 0
  if simp:
    eq = simplify(eq)
  return eq

deltaSpotBound = {asset: deltaSpotBound_(asset, *all_symbols[asset]) for asset in assets}
deltaSpotBound['b']

-Delta_b - b_b + s_b/w_b - v_b

In [9]:
deltaSpotBound["s"];

-Delta_s - b_s + s_s/w_s - v_s

In [10]:
def spotExpBound_(asset, s,v, b, w, j, e, Delta, a, min):
  eq = spotByExp[asset] - e # ==! 0
  if simp:
    eq = simplify(eq)
  return eq

spotExpBound = {asset: spotExpBound_(asset, *all_symbols[asset]) for asset in assets}
spotExpBound['b']

a_b*(j_b + 1)**e_b/j_b**e_b - e_b

In [11]:
spotExpBound["s"];

a_s*(j_s + 1)**e_s/j_s**e_s - e_s

## Inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

### value cannot decrease

In [12]:
a0Buying = s_s * Delta_b # < 0 if delta_b_negative, > 0 otherwise
if delta_b_negative:
    a0Buying = -a0Buying # > 0
a0Buying

Delta_b*s_s

In [13]:
a0Selling = s_b * Delta_s # > 0
a0Selling

Delta_s*s_b

In [14]:
a0Bound = a0Buying - a0Selling # <=! 0
a0Bound

Delta_b*s_s - Delta_s*s_b

In [15]:
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

Delta_b*s_s - Delta_s*s_b

### exponents cannot result in spot prices that are better than the amm-prices

In [16]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [17]:
expBound["s"];

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

### spot prices cannot exceed maxInteger

In [18]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(s, v, b, w, j, e, Delta, a, min):
  return s - I_max # <=! 0

maxSpotBound = {asset: maxSpotBound_(*all_symbols[asset]) for asset in assets}
maxSpotBound['b']

-I_max + s_b

In [19]:
maxSpotBound["s"];

-I_max + s_s

### need to swap at least minimum amounts

In [20]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
  if delta_b_negative and asset == 'b':
    return min + Delta # <=! 0
  else:
    return min - Delta # <=! 0

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

-Delta_b + min_b

In [21]:
minAmntBound["s"];

-Delta_s + min_s

### need to swap at most maximum amounts

In [22]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
  if asset == 'b':
    if delta_b_negative:
      return -Delta - b # <=! 0
    else:
      return Delta - b # <=! 0
  else:
    return Delta - max_s # <=! 0

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

Delta_b - b_b

### total number of multiplications of both exponentiations cannot exceed expLimit

In [23]:
# expLimit, aka maximum total number of multiplications for both exponents
m_max = symbols('m_max', positive=True, integer=True)

def numMultsLower_(asset, s,v, b, w, j, e, Delta, a, min):
  return floor(log(e, 2))

# the best number of multiplications we can get by increasing e
def bestMultsAhead_(asset, s,v, b, w, j, e, Delta, a, min):
  return ceiling(log(e, 2))

bestMultsAhead = {asset: bestMultsAhead_(asset, *all_symbols[asset]) for asset in assets}
bestMultsAhead['b']

ceiling(log(e_b)/log(2))

In [24]:
# optimistic estimate. This relies on the vague intuition that exponents will be minimal from the other constraints, so best we can do is increase them if this one is violated
optimisticMultsBound = bestMultsAhead['b'] + bestMultsAhead['s'] - m_max # <=! 0
if simp:
  optimisticMultsBound = simplify(optimisticMultsBound)
optimisticMultsBound

-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))

## Lagrangian

### per asset:

In [25]:
def custom_diff(f, n):
    diff = f.diff(n)
    # diff = floor(f.subs(n, n+1) - f)
    if simp:
        diff = simplify(diff)
    return diff

def equations(asset, s,v, b, w, j, e, Delta, a, min):
  
    # Decision variables
    xs = [e, s, Delta]

    # Equality-Constraints
    hs = [
        deltaSpotBound[asset],
        spotExpBound[asset],
    ] 

    # Inequality-Constraints
    gs = [
        expBound[asset],
        maxSpotBound[asset],
        minAmntBound[asset],
        maxAmntBound[asset],
        # bestMultsAhead[asset] - m_max,
    ] 
    
    # Lagrange multipliers for equality- and inequality-constraints
    mus = symbols('mu^{}_1:{}'.format(asset, len(hs)+1))
    lambdas = symbols('lambda^{}_1:{}'.format(asset, len(gs)+1), nonnegative=True)

    # Objective function
    f = eff

    # Lagrangian
    L = f\
        + sum([mu * h for mu, h in zip(mus, hs)])\
        + sum([lambda_ * g for lambda_, g in zip(lambdas, gs)])

    diffs = []
    print("derivative equations:")
    for x in xs:
        dL_dx = custom_diff(L, x)
        equation = Eq(dL_dx, 0)
        if simp: # results in error from the bestMultsAhead-constraint
          equation = simplify(equation)
        display(equation)
        diffs.append(equation)

    primal = []
    # Primal feasibility for equality-constraints
    print("primal feasibility for equality-constraints:")
    for h in hs:
        equation = Eq(h, 0)
        if simp:
            equation = simplify(equation)
        display(equation)
        primal.append(equation)

    slack = []
    # Complementary slackness conditions
    print("complementary slackness conditions:")
    for lambda_, g in zip(lambdas, gs):
        equation = Eq(lambda_ * g, 0)
        if simp:
            equation = simplify(equation)
        display(equation)
        slack.append(equation)

    return diffs, primal, slack, hs, gs, xs, list(mus), list(lambdas)

In [26]:
diffs_b, primal_b, slack_b, hs_b, gs_b, xs_b, mus_b, lambdas_b = equations('b', *all_symbols['b'])
equations_b = diffs_b + primal_b + slack_b
vars_b = xs_b + mus_b + lambdas_b

derivative equations:


Eq(-lambda^b_1 + mu^b_2 + mu^b_2*log(j_b**(a_b*(j_b + 1)**e_b)/(j_b + 1)**(a_b*(j_b + 1)**e_b))/j_b**e_b, 0)

Eq(lambda^b_2 + mu^b_1/w_b, 0)

Eq(lambda^b_3 - lambda^b_4 + mu^b_1 + Delta_s/Delta_b**2, 0)

primal feasibility for equality-constraints:


...

complementary slackness conditions:


Eq(lambda^b_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))), 0)

Eq(lambda^b_2*(-I_max + s_b), 0)

Eq(lambda^b_3*(-Delta_b + min_b), 0)

Eq(lambda^b_4*(Delta_b - b_b), 0)

In [27]:
diffs_s, primal_s, slack_s, hs_s, gs_s, xs_s, mus_s, lambdas_s = equations('s', *all_symbols['s'])
equations_s = diffs_s + primal_s + slack_s
vars_s = xs_s + mus_s + lambdas_s

derivative equations:


Eq(lambda^s_1 + mu^s_2 + mu^s_2*log(j_s**(a_s*(j_s + 1)**e_s)/(j_s + 1)**(a_s*(j_s + 1)**e_s))/j_s**e_s, 0)

Eq(lambda^s_2 + mu^s_1/w_s, 0)

Eq(lambda^s_3 - lambda^s_4 + mu^s_1 - 1/Delta_b, 0)

primal feasibility for equality-constraints:


...

complementary slackness conditions:


Eq(lambda^s_1*(e_s - log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))), 0)

Eq(lambda^s_2*(-I_max + s_s), 0)

Eq(lambda^s_3*(-Delta_s + min_s), 0)

Eq(lambda^s_4*(Delta_s - max_s), 0)

In [28]:
def verify_inequality_constraints(solution, vars, gs):
    subs_dict = dict(zip(vars, solution))
    for g in gs:
        if Gt(g.subs(subs_dict), 0) == True:
            return False
    return True

In [29]:
solutions_b = nonlinsolve(equations_b, vars_b);
len(solutions_b);

8

In [30]:
solutions_b = [solution for solution in solutions_b if verify_inequality_constraints(solution, vars_b, gs_b)]
len(solutions_b)

8

In [31]:
vars_b;

[e_b,
 s_b,
 Delta_b,
 mu^b_1,
 mu^b_2,
 lambda^b_1,
 lambda^b_2,
 lambda^b_3,
 lambda^b_4]

In [32]:
for solution in solutions_b:
    display(solution)

(e_b, w_b*(2*b_b + v_b), b_b, 0, 0, 0, 0, 0, Delta_s/b_b**2)

(e_b, w_b*(b_b + min_b + v_b), min_b, 0, 0, 0, 0, -Delta_s/min_b**2, 0)

(e_b, I_max, I_max/w_b - b_b - v_b, -Delta_s*w_b**2/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), 0, 0, Delta_s*w_b/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), 0, 0)

(e_b, -I_max + 3*b_b*w_b + min_b*w_b + 2*v_b*w_b, -I_max/w_b + 2*b_b + min_b + v_b, Delta_s*w_b**2/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), 0, 0, -Delta_s*w_b/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), -Delta_s/min_b**2, Delta_s/b_b**2)

(log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))), w_b*(2*b_b + v_b), b_b, 0, mu^b_2, mu^b_2 + mu^b_2*log(j_b**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))))/(j_b + 1)**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1))))))/j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))), 0, 0, Delta_s/b_b**2)

(log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))), w_b*(b_b + min_b + v_b), min_b, 0, mu^b_2, mu^b_2 + mu^b_2*log(j_b**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))))/(j_b + 1)**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1))))))/j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))), 0, -Delta_s/min_b**2, 0)

(log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))), I_max, I_max/w_b - b_b - v_b, -Delta_s*w_b**2/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), mu^b_2, mu^b_2 + mu^b_2*log(j_b**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))))/(j_b + 1)**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1))))))/j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))), Delta_s*w_b/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), 0, 0)

(log((a_b/(w_b*(b_b + v_b)))**(1/(log(j_b) - log(j_b + 1)))), -I_max + 3*b_b*w_b + min_b*w_b + 2*v_b*w_b, -I_max/w_b + 2*b_b + min_b + v_b, Delta_s*w_b**2/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), mu^b_2, mu^b_2 + mu^b_2*log(j_b**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))))/(j_b + 1)**(a_b*(j_b + 1)**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1))))))/j_b**log(a_b**(1/(log(j_b) - log(j_b + 1)))/(b_b*w_b + v_b*w_b)**(1/(log(j_b) - log(j_b + 1)))), -Delta_s*w_b/(I_max**2 - 2*I_max*b_b*w_b - 2*I_max*v_b*w_b + b_b**2*w_b**2 + 2*b_b*v_b*w_b**2 + v_b**2*w_b**2), -Delta_s/min_b**2, Delta_s/b_b**2)

In [33]:
solutions_s = nonlinsolve(equations_s, vars_s);
len(solutions_s);

8

In [34]:
solutions_s = [solution for solution in solutions_s if verify_inequality_constraints(solution, vars_s, gs_s)]
len(solutions_s)

8

In [35]:
vars_s;

[e_s,
 s_s,
 Delta_s,
 mu^s_1,
 mu^s_2,
 lambda^s_1,
 lambda^s_2,
 lambda^s_3,
 lambda^s_4]

In [36]:
for solution in solutions_s:
    display(solution)

(e_s, w_s*(b_s + min_s + v_s), min_s, 0, 0, 0, 0, 1/Delta_b, 0)

(e_s, w_s*(b_s + max_s + v_s), max_s, 0, 0, 0, 0, 0, -1/Delta_b)

(e_s, I_max, I_max/w_s - b_s - v_s, 1/Delta_b, 0, 0, -1/(Delta_b*w_s), 0, 0)

(e_s, s_s, -b_s + s_s/w_s - v_s, mu^s_1, 0, 0, -mu^s_1/w_s, (Delta_b*I_max*mu^s_1 - Delta_b*b_s*mu^s_1*w_s - Delta_b*max_s*mu^s_1*w_s - Delta_b*mu^s_1*v_s*w_s + b_s*w_s + max_s*w_s - s_s + v_s*w_s)/(Delta_b*w_s*(max_s - min_s)), (Delta_b*I_max*mu^s_1 - Delta_b*b_s*mu^s_1*w_s - Delta_b*min_s*mu^s_1*w_s - Delta_b*mu^s_1*v_s*w_s + b_s*w_s + min_s*w_s - s_s + v_s*w_s)/(Delta_b*w_s*(max_s - min_s)))

(log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))), w_s*(b_s + min_s + v_s), min_s, 0, mu^s_2, -mu^s_2 - mu^s_2*log(j_s**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))))/(j_s + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1))))))/j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))), 0, 1/Delta_b, 0)

(log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))), w_s*(b_s + max_s + v_s), max_s, 0, mu^s_2, -mu^s_2 - mu^s_2*log(j_s**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))))/(j_s + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1))))))/j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))), 0, 0, -1/Delta_b)

(log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))), I_max, I_max/w_s - b_s - v_s, 1/Delta_b, mu^s_2, -mu^s_2 - mu^s_2*log(j_s**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))))/(j_s + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1))))))/j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))), -1/(Delta_b*w_s), 0, 0)

(log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))), s_s, -b_s + s_s/w_s - v_s, mu^s_1, mu^s_2, -mu^s_2 - mu^s_2*log(j_s**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))))/(j_s + 1)**(a_s*(j_s + 1)**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1))))))/j_s**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(b_s*w_s + v_s*w_s)**(1/(log(j_s) - log(j_s + 1)))), -mu^s_1/w_s, (Delta_b*I_max*mu^s_1 - Delta_b*b_s*mu^s_1*w_s - Delta_b*max_s*mu^s_1*w_s - Delta_b*mu^s_1*v_s*w_s + b_s*w_s + max_s*w_s - s_s + v_s*w_s)/(Delta_b*w_s*(max_s - min_s)), (Delta_b*I_max*mu^s_1 - Delta_b*b_s*mu^s_1*w_s - Delta_b*min_s*mu^s_1*w_s - Delta_b*mu^s_1*v_s*w_s + b_s*w_s + min_s*w_s - s_s + v_s*w_s)/(Delta_b*w_s*(max_s - min_s)))

### with both assets:

In [37]:
primal_equations = primal_b + primal_s
primal_vars = mus_b + mus_s
for equation in primal_equations:
  display(equation)

Eq(Delta_b + b_b - s_b/w_b + v_b, 0)

Eq(a_b*(j_b + 1)**e_b/j_b**e_b - e_b, 0)

Eq(Delta_s + b_s - s_s/w_s + v_s, 0)

Eq(a_s*(j_s + 1)**e_s/j_s**e_s - e_s, 0)

In [38]:
primal_vars;

[mu^b_1, mu^b_2, mu^s_1, mu^s_2]

In [39]:
slack_equations = slack_b + slack_s
slack_vars = lambdas_b + lambdas_s
for equation in slack_equations:
  display(equation)

Eq(lambda^b_1*(e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))), 0)

Eq(lambda^b_2*(-I_max + s_b), 0)

Eq(lambda^b_3*(-Delta_b + min_b), 0)

Eq(lambda^b_4*(Delta_b - b_b), 0)

Eq(lambda^s_1*(e_s - log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))), 0)

Eq(lambda^s_2*(-I_max + s_s), 0)

Eq(lambda^s_3*(-Delta_s + min_s), 0)

Eq(lambda^s_4*(Delta_s - max_s), 0)

In [40]:
gs = [ # inequality-constraints which involve both assets
  a0Bound,
  optimisticMultsBound,
]
lambdas = symbols('lambda_1:{}'.format(len(gs)+1), nonnegative=True)
for lambda_, g in zip(lambdas, gs):
    equation = Eq(lambda_ * g, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    slack_equations.append(equation)
    slack_vars.append(lambda_)


Eq(lambda_1*(Delta_b*s_s - Delta_s*s_b), 0)

Eq(lambda_2*(-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))), 0)

In [41]:
slack_vars;

[lambda^b_1,
 lambda^b_2,
 lambda^b_3,
 lambda^b_4,
 lambda^s_1,
 lambda^s_2,
 lambda^s_3,
 lambda^s_4,
 lambda_1,
 lambda_2]

In [42]:
# Objective function
f = eff

diff_equations = []
diff_vars = xs_b + xs_s

# Lagrangian
L = f\
    + sum([mu * h for mu, h in zip(hs_b + hs_s, mus_b + mus_s)])\
    + sum([lambda_ * g for lambda_, g in zip(gs_b + gs_s + gs, lambdas_b + lambdas_s + list(lambdas))])

for x in xs_b + xs_s:
    dL_dx = custom_diff(L, x)
    equation = Eq(dL_dx, 0)
    # if simp: # results in error from the bestMultsAhead-constraint
    #   equation = simplify(equation)
    display(equation)
    diff_equations.append(equation)

Eq((j_b**e_b*lambda_2*Subs(Derivative(ceiling(_xi_1), _xi_1), _xi_1, log(e_b)/log(2)) - mu^b_2*log(2**(e_b*(j_b**e_b + log(j_b**(a_b*(j_b + 1)**e_b)/(j_b + 1)**(a_b*(j_b + 1)**e_b))))) + log(2**(e_b*j_b**e_b*lambda^b_1)))/(e_b*j_b**e_b*log(2)), 0)

Eq(-Delta_s*lambda_1 + lambda^b_2 + mu^b_1/w_b, 0)

Eq(-lambda^b_3 + lambda^b_4 + lambda_1*s_s - mu^b_1 - Delta_s/Delta_b**2, 0)

Eq((j_s**e_s*lambda_2*Subs(Derivative(ceiling(_xi_1), _xi_1), _xi_1, log(e_s)/log(2)) - mu^s_2*log(2**(e_s*(j_s**e_s + log(j_s**(a_s*(j_s + 1)**e_s)/(j_s + 1)**(a_s*(j_s + 1)**e_s))))) - log(2**(e_s*j_s**e_s*lambda^s_1)))/(e_s*j_s**e_s*log(2)), 0)

Eq(Delta_b*lambda_1 + lambda^s_2 + mu^s_1/w_s, 0)

Eq(-lambda^s_3 + lambda^s_4 - lambda_1*s_b - mu^s_1 + 1/Delta_b, 0)

In [43]:
diff_vars;

[e_b, s_b, Delta_b, e_s, s_s, Delta_s]

In [44]:
equations = diff_equations + primal_equations + slack_equations;
vars = diff_vars + primal_vars + slack_vars;

In [45]:
final_solutions = []

def print_solution(solution, vars):
  strs = []
  for var, expression in zip(vars, solution):
    strs.append(f"{var} = {expression}")
  print(", ".join(strs))

for i_b, solution_b in enumerate(solutions_b):
  print(i_b, "of", len(solutions_b))
  subs_dict_b = dict(zip(vars_b, list(solution_b)))
  for i_s, solution_s in enumerate(solutions_s):
    subs_dict_s = dict(zip(vars_s, list(solution_s)))
    pair_equations = []
    pair_vars = []
    for equation, var in zip(equations, vars):
      eq = equation.subs(subs_dict_b).subs(subs_dict_s)
      # display(eq)
      if simp:
        eq = simplify(eq)
      if eq != True:
        novel = True
        for eq_ in pair_equations:
          if eq_ == eq:
            novel = False
            break
        if novel:
          pair_vars.append(var)
          pair_equations.append(eq)
    pair_solutions = nonlinsolve(pair_equations, pair_vars)
    for found_solution in pair_solutions:
      novel = True
      for _, known_solution in final_solutions:
        if known_solution == found_solution:
          novel = False
          break
      if novel:
        print_solution(found_solution, pair_vars)
        final_solutions.append((pair_vars, found_solution))

0 of 8
e_b = e_b, s_b = s_b, Delta_b = b_b, e_s = e_s, s_s = s_s, Delta_s = Delta_s, mu^b_2 = mu^b_2, mu^s_2 = mu^s_2, lambda_1 = 0, lambda_2 = 0
1 of 8
e_b = e_b, s_b = s_b, Delta_b = min_b, e_s = e_s, s_s = s_s, Delta_s = Delta_s, mu^b_2 = mu^b_2, mu^s_2 = mu^s_2, lambda_1 = 0, lambda_2 = 0
2 of 8
e_b = e_b, s_b = s_b, Delta_b = I_max/w_b - b_b - v_b, e_s = e_s, s_s = s_s, Delta_s = Delta_s, mu^b_2 = mu^b_2, mu^s_2 = mu^s_2, lambda_1 = 0, lambda_2 = 0
3 of 8
4 of 8
5 of 8
6 of 8
7 of 8


In [46]:
final_solutions = [(vars, solution) for vars, solution in final_solutions if verify_inequality_constraints(solution, vars, gs_b + gs_s + gs)]
len(final_solutions)

3

In [47]:
def display_solution(solution, vars):
  for var, expression in zip(vars, solution):
    if var != expression and expression != 0:
      display(Eq(var, expression))

valid_assignments = []
for solution in final_solutions:
  valid = True
  assignments = []
  for var, expression in zip(solution[0], solution[1]):
    if expression == 0:
      if var in [s_b, s_s, Delta_b, Delta_s]:
        valid = False
        break
      elif not var in [e_b, e_s]:
        continue
    if var != expression:
      assignments.append(Eq(var, expression))
  if not valid:
    continue
  if len(assignments) == 0:
    continue
  assert(len(assignments) == 1) # just an observation we are relying upon for this display
  assignment = assignments[0]
  if simp:
    assignment = simplify(assignment)
  if assignment in valid_assignments:
    continue
  valid_assignments.append(assignment)
  # print(assignment)
  display(assignment)

Eq(Delta_b, b_b)

Eq(Delta_b, min_b)

Eq(Delta_b, I_max/w_b - b_b - v_b)